In [1]:
import pandas as pd
import numpy as np
import re, glob,os,sys,pickle,random
from collections import defaultdict

In [2]:
#!pip install ase
#!git clone https://github.com/dmamur/elementsem.git

In [3]:
os.getcwd()

'/home/dbsejrgus226/FPR2-Drug-Discovery/elemBERT_custom/elembert'

In [4]:
from ase.io import read,write
from scipy.ndimage import gaussian_filter1d
from ase.neighborlist import neighbor_list
from joblib import dump, load
from sklearn.cluster import KMeans
from sklearn import decomposition
from ase import Atoms

ellst=open('./models/pcakm/ellist.txt','r').read().split('\n')
print(ellst)

km = {i: load('./models/pcakm/'+i+'_kmeans.pkl') for i in ellst}
pca = {i: load('./models/pcakm/'+i+'_pca.pkl') for i in ellst}

def getRawInputs(types,atoms,x,v):
    i, d = neighbor_list('id', atoms, 10.0, self_interaction=False)
    rdfatoms,ntypes=[],[]
    for k,l in enumerate(atoms):
        el=types[k]
        y = np.zeros(100)
        dist = np.round(d[i==k]*10)
        a,b=np.unique(dist, return_counts=True)
        np.put(y,a.astype(int)-1,b)
        values=gaussian_filter1d(y/v,1)
        num = km[el].predict(pca[el].transform(np.nan_to_num([values],nan=0,posinf=0, neginf=0)))[0]
        ntypes.append(el+str(num))#el2id[el+str(num)]
    return ntypes

['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Rn']


/home/dbsejrgus226/miniconda3/envs/elembert/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator KMeans from version 1.1.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/dbsejrgus226/miniconda3/envs/elembert/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.1.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
x= np.arange(0,10,0.1)
v = np.concatenate([[1],4*np.pi/3*(x[1:]**3 - x[:-1]**3)])

In [6]:
atoms = pd.read_csv("./data/final_tanimoto_scoring_data.csv", usecols=["smiles"])
atoms.head()

,smiles
0,COc1ccc(NC(=O)N[C@@H]2CCN(c3ccccc3F)C2=O)cc1
1,COc1ccc(NC(=O)N[C@H]2CCN(c3ccccc3F)C2=O)cc1
2,O=C(NCc1cccnc1Oc1ccc(F)cc1F)c1c[nH]c(=O)[nH]1
3,O=C1C[C@H](NC(=O)Nc2ccc(Oc3cccc(F)c3F)nc2)CN1
4,COc1cc(F)ccc1NC(=O)N[C@@H](C)C(=O)NCc1ccco1


In [7]:
len(atoms)

43

In [8]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors

from tqdm import tqdm

### SMLES-> molecule(화학식)으로 변환하는 과정

In [9]:
params = AllChem.ETKDGv3()
params.randomSeed = 0xf00d

for i in tqdm(range(len(atoms)), desc="Generating mol files..."):
    
    mol = Chem.MolFromSmiles(atoms["smiles"][i])
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, params)
    results = Chem.MolToMolBlock(mol)
    print(results, file=open(f"./data/mol_file_{i}.mol", "w+"))

Generating mol files...: 100%|██████████| 43/43 [00:00<00:00, 49.69it/s]


In [10]:
types_lst = []
for i in tqdm(range(len(atoms)), desc="Generating..."):
    atom = read(f"./data/mol_file_{i}.mol", format="mol")
    types = getRawInputs(atom.get_chemical_symbols(), atom, x, v)
    types_lst.append(types)

atoms["typesV1"] = types_lst

Generating...: 100%|██████████| 43/43 [00:00<00:00, 53.60it/s]


In [11]:
atoms.head()

,smiles,typesV1
0,COc1ccc(NC(=O)N[C@@H]2CCN(c3ccccc3F)C2=O)cc1,"[C1, O0, C0, C3, C3, C0, N0, C0, O0, N0, C3, C..."
1,COc1ccc(NC(=O)N[C@H]2CCN(c3ccccc3F)C2=O)cc1,"[C1, O0, C0, C3, C3, C0, N7, C0, O0, N0, C3, C..."
2,O=C(NCc1cccnc1Oc1ccc(F)cc1F)c1c[nH]c(=O)[nH]1,"[O0, C3, N7, C5, C0, C3, C3, C3, N1, C0, O0, C..."
3,O=C1C[C@H](NC(=O)Nc2ccc(Oc3cccc(F)c3F)nc2)CN1,"[O0, C0, C5, C3, N0, C0, O0, N0, C0, C3, C3, C..."
4,COc1cc(F)ccc1NC(=O)N[C@@H](C)C(=O)NCc1ccco1,"[C1, O0, C0, C3, C0, F1, C3, C3, C0, N7, C3, O..."


In [12]:
atoms.to_csv("./data/custom_data.csv", index=False)

In [13]:
len(atoms)

43